# Time Series Modeling (ARIMA Methods) Assignment

In [1]:
%reload_ext nb_black
import pandas as pd
import plotly.express as px
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

<IPython.core.display.Javascript object>

In [10]:
def ilinechart(df, x, y, groups=None, title=""):
    fig = px.line(df, x=x, y=y, color=groups, title=title, template="none").update(
        layout=dict(title=dict(x=0.5))
    )

    fig.show()

<IPython.core.display.Javascript object>

### Import the Amazon stock price data set (AMZN_data.csv) and plot a line chart with the observed daily closing prices.

In [2]:
amazon = pd.read_csv(
    "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Time%20Series%20Analysis/Time%20Series%20-%20Day%204/AMZN_data.csv"
)
amazon.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,261.40,265.25,260.555,261.95,3879078,AMZN
1,2013-02-11,263.20,263.25,256.600,257.21,3403403,AMZN
2,2013-02-12,259.19,260.16,257.000,258.70,2938660,AMZN
3,2013-02-13,261.53,269.96,260.300,269.47,5292996,AMZN
4,2013-02-14,267.37,270.65,265.400,269.24,3462780,AMZN


<IPython.core.display.Javascript object>

In [3]:
amazon["date"] = pd.to_datetime(amazon["date"])

<IPython.core.display.Javascript object>

In [11]:
ilinechart(amazon, "date", "close", title="Closing Prices")

<IPython.core.display.Javascript object>

### Run an Autoregression (AR) model on the series and add the results to a column in the dataframe.

In [6]:
model = AR(amazon["close"]).fit()
amazon["AR"] = model.predict()

<IPython.core.display.Javascript object>

### Plot a multi-line chart comparing the AR model's results with the observed values.

In [12]:
melted = pd.melt(
    amazon,
    id_vars="date",
    value_vars=["close", "AR",],
    var_name="Variable",
    value_name="Value",
)

<IPython.core.display.Javascript object>

In [13]:
melted.head()

,date,Variable,Value
0,2013-02-08,close,261.95
1,2013-02-11,close,257.21
2,2013-02-12,close,258.70
3,2013-02-13,close,269.47
4,2013-02-14,close,269.24


<IPython.core.display.Javascript object>

In [17]:
ilinechart(melted, "date", "Value", groups="Variable")

<IPython.core.display.Javascript object>

### Run a Moving Average (MA) model on the series and add the results to a column in the dataframe.

In [18]:
model = ARMA(amazon["close"], order=(0, 1)).fit()
amazon["MA"] = model.predict()

<IPython.core.display.Javascript object>

### Print the model summary and add the model's results to the line chart with observations and the AR model results.

In [19]:
print(model.summary())

                              ARMA Model Results                              
Dep. Variable:                  close   No. Observations:                 1259
Model:                     ARMA(0, 1)   Log Likelihood               -8047.769
Method:                       css-mle   S.D. of innovations            144.300
Date:                Thu, 29 Oct 2020   AIC                          16101.538
Time:                        10:39:13   BIC                          16116.952
Sample:                             0   HQIC                         16107.331
                                                                              
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const         576.9810      8.063     71.561      0.000     561.178     592.784
ma.L1.close     0.9834      0.004    226.362      0.000       0.975       0.992
                                    Roots       

<IPython.core.display.Javascript object>

In [20]:
melted = pd.melt(
    amazon,
    id_vars="date",
    value_vars=["close", "AR", "MA"],
    var_name="Variable",
    value_name="Value",
)

<IPython.core.display.Javascript object>

In [21]:
ilinechart(melted, "date", "Value", groups="Variable")

<IPython.core.display.Javascript object>

### Run an Autoregressive Moving Average (ARMA) model on the series and add the results to a column in the dataframe.

Hint: You will need to make the time series stationary in order to run the ARMA model on it. The most common way to do this is by differncing, or subtracting the previous observed value from the current one.

In [22]:
amazon["diff_1"] = amazon.close - amazon.close.shift()

<IPython.core.display.Javascript object>

In [32]:
model = ARMA(amazon["diff_1"][1:], order=(1, 1)).fit()
amazon["ARMA"] = amazon["close"] + model.predict()

<IPython.core.display.Javascript object>

### Print the model summary and generate a multi-line chart that compares the ARMA model's results with those from the AR and MA models.

Hint: You will need to de-difference the model results by adding the previous observed values to them.

In [33]:
print(model.summary())

                              ARMA Model Results                              
Dep. Variable:                 diff_1   No. Observations:                 1258
Model:                     ARMA(1, 1)   Log Likelihood               -4734.196
Method:                       css-mle   S.D. of innovations             10.426
Date:                Thu, 29 Oct 2020   AIC                           9476.392
Time:                        10:54:11   BIC                           9496.941
Sample:                             0   HQIC                          9484.115
                                                                              
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.9192      0.290      3.173      0.002       0.351       1.487
ar.L1.diff_1    -0.6763      0.277     -2.441      0.015      -1.219      -0.133
ma.L1.diff_1     0.6518      0.283      2.30

<IPython.core.display.Javascript object>

In [34]:
melted = pd.melt(
    amazon,
    id_vars="date",
    value_vars=["close", "AR", "MA", "ARMA"],
    var_name="Variable",
    value_name="Value",
)

<IPython.core.display.Javascript object>

In [35]:
ilinechart(melted, "date", "Value", groups="Variable")

<IPython.core.display.Javascript object>

### Run an Autoregressive Integrated Moving Average (ARIMA) model on the series and add the results to a column in the dataframe.

In [36]:
model = ARIMA(amazon["close"], order=(1, 1, 1)).fit()
amazon["ARIMA"] = model.predict(typ="levels")

<IPython.core.display.Javascript object>

### Print the model summary and generate a multi-line chart that compares the ARIMA model's results to the observed values.

In [37]:
print(model.summary())

                             ARIMA Model Results                              
Dep. Variable:                D.close   No. Observations:                 1258
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -4734.196
Method:                       css-mle   S.D. of innovations             10.426
Date:                Thu, 29 Oct 2020   AIC                           9476.392
Time:                        10:55:44   BIC                           9496.941
Sample:                             1   HQIC                          9484.115
                                                                              
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.9192      0.290      3.173      0.002       0.351       1.487
ar.L1.D.close    -0.6763      0.277     -2.441      0.015      -1.219      -0.133
ma.L1.D.close     0.6518      0.283     

<IPython.core.display.Javascript object>

In [38]:
melted = pd.melt(
    amazon,
    id_vars="date",
    value_vars=["close", "AR", "MA", "ARMA", "ARIMA"],
    var_name="Variable",
    value_name="Value",
)

<IPython.core.display.Javascript object>

In [39]:
ilinechart(melted, "date", "Value", groups="Variable")

<IPython.core.display.Javascript object>

### Run an Seasonal Autoregressive Integrated Moving Average (SARIMA) model on the series and add the results to a column in the dataframe.

In [41]:
model = SARIMAX(amazon["close"], order=(1, 1, 1), seasonal_order=(1, 1, 1, 2)).fit()
amazon["SARIMA"] = model.predict()

<IPython.core.display.Javascript object>

### Print the model summary and generate a multi-line chart that compares the SARMA model's results with the observed values.

### Run an Seasonal Autoregressive Integrated Moving Average with Exogenous Factors (SARIMAX) model on the series and add the results to a column in the dataframe.

Use the daily Open prices as the exogenous factors.

### Print the model summary and generate a multi-line chart that compares the SARMAX model's results with the SARIMA model's values.

### Evaluate the performance of all the models and compare to each other, using Mean Absolute Error and Root Mean Squared Error as your evaluation metrics.